**Add features on top of BERT fine tunning, see reference**

https://datascience.stackexchange.com/questions/54888/how-can-i-add-custom-numerical-features-for-training-to-bert-fine-tuning

https://www.pyimagesearch.com/2019/02/04/keras-multiple-inputs-and-mixed-data/

Binary_crossentropy: https://gombru.github.io/2018/05/23/cross_entropy_loss/

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install transformers

     |████████████████████████████████| 665kB 3.4MB/s 
     |████████████████████████████████| 890kB 15.6MB/s 
     |████████████████████████████████| 1.1MB 26.9MB/s 
     |████████████████████████████████| 3.8MB 29.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=cf464331f058fcd0209973841517682d17334a2350a220adb37bca700b745da0
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
ROOT_PATH = '/content/drive/My Drive/Qishi/NLP/project'
# ROOT_PATH = '/kaggle'

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk(ROOT_PATH+'/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/content/drive/My Drive/Qishi/NLP/project/input/tweet-sentiment-extraction/train.csv
/content/drive/My Drive/Qishi/NLP/project/input/tweet-sentiment-extraction/sample_submission.csv
/content/drive/My Drive/Qishi/NLP/project/input/tweet-sentiment-extraction/test.csv
/content/drive/My Drive/Qishi/NLP/project/input/tfrobert/glove_6B_100d_top100k.csv
/content/drive/My Drive/Qishi/NLP/project/input/tfrobert/roberta-base-vocab.json
/content/drive/My Drive/Qishi/NLP/project/input/tfrobert/roberta-base-merges.txt
/content/drive/My Drive/Qishi/NLP/project/input/tfrobert/pretrained-roberta-base.h5
/content/drive/My Drive/Qishi/NLP/project/input/tfrobert/roberta-base-tf_model.h5
/content/drive/My Drive/Qishi/NLP/project/input/tfrobert/config-roberta-base.json
/content/drive/My Drive/Qishi/NLP/project/input/tfrobert/roberta-base-config.json
/content/drive/My Drive/Qishi/NLP/project/input/roberta-base/pytorch_model.bin
/content/drive/My Drive/Qishi/NLP/project/input/roberta-base/config.json
/conten

In [5]:
import pandas as pd, numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold, ShuffleSplit
from transformers import *
import tokenizers
import gc
from tensorflow.python.framework import ops
print('TF version',tf.__version__)

TF version 2.2.0


In [0]:
a = np.array([[0, 1], [0, 1]])
b = np.array([[0, 1], [0, 1]])
ab = np.concatenate([a, b], axis=-1)
c = np.array([[0.2, 0.8], [0.2, 0.8]])
d= np.array([[0, 1.0], [0, 1.0]])
cd = np.concatenate([c, d], axis=-1)

In [16]:
ab = Concatenate(axis=-1)([a, b])
cd = Concatenate(axis=-1)([c, d])


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [18]:
tf.keras.losses.CategoricalCrossentropy()(K.cast(ab[:, :2], dtype='float32'), cd[:,:2])

<tf.Tensor: shape=(), dtype=float32, numpy=0.22314353>

In [0]:
from tensorflow.keras.layers import Input, Dropout, Conv1D, LeakyReLU, Dense, Flatten, Activation, Reshape, Concatenate, Bidirectional, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

learning_rate_fn = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=2e-5,
    decay_steps=10000,
    end_learning_rate=1e-6,
    power=1)

def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    if (len(a)==0) & (len(b)==0): return 0.5
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))


class FineTuneBertForQA:
    def __init__(self, tokenizer, config_file, bert_model_file, max_length, n_splits, loss_fn='categorical_crossentropy', separated_output=True, seed=777):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.sentiment_id = {s:self.tokenizer.encode(s).ids[0] for s in ['positive', 'negative', 'neutral']}
        self.config_file = config_file
        self.bert_model_file = bert_model_file
        self.n_splits = n_splits
        self.loss_fn = loss_fn
        self.separated_output = separated_output
        self.seed = seed

    def bert_data_transform(self, data, train=True):
        '''
        Transform data into arrays that BERT understands 
        '''
        ct = data.shape[0]
        input_ids = np.ones((ct,self.max_length),dtype='int32')
        attention_mask = np.zeros((ct,self.max_length),dtype='int32')
        #token_type_ids = np.zeros((ct,self.max_length),dtype='int32')
        if train:
            if self.separated_output:
                start_tokens = np.zeros((ct,self.max_length),dtype='int32')
                end_tokens = np.zeros((ct,self.max_length),dtype='int32')
            else:
                span_tokens = np.zeros((ct,MAX_LEN),dtype='int32')

        for k in range(ct):
            # FIND OVERLAP
            text1 = " "+" ".join(data.loc[k,'text'].split())
            enc = self.tokenizer.encode(text1)
            s_tok = self.sentiment_id[data.loc[k,'sentiment']]
            input_ids[k,:len(enc.ids)+5] = [0] + enc.ids + [2,2] + [s_tok] + [2]
            attention_mask[k,:len(enc.ids)+5] = 1
            
            if train:
                text2 = " ".join(data.loc[k,'selected_text'].split())
                idx = text1.find(text2)
                chars = np.zeros((len(text1)))
                chars[idx:idx+len(text2)]=1
                if text1[idx-1]==' ': chars[idx-1] = 1 
                # ID_OFFSETS
                offsets = []; idx=0
                for t in enc.ids:
                    w = self.tokenizer.decode([t])
                    offsets.append((idx,idx+len(w)))
                    idx += len(w)

                # START END TOKENS
                toks = []
                for i,(a,b) in enumerate(offsets):
                    sm = np.sum(chars[a:b])
                    if sm>0: toks.append(i) 

                if len(toks)>0:
                    if self.separated_output:
                        start_tokens[k,toks[0]+1] = 1
                        end_tokens[k,toks[-1]+1] = 1
                    else:
                        for i in range(toks[0], toks[-1]+1):
                            span_tokens[k,i+1] = 1

        if train:
            if self.separated_output:
                return (input_ids, attention_mask, start_tokens, end_tokens)
            else:
                return (input_ids, attention_mask, span_tokens)
        else:
            return (input_ids, attention_mask)
    
    def build_model(self):
        '''
        Add layer on top of BERT
        '''        
        ids = Input((self.max_length,), dtype=tf.int32)
        att = Input((self.max_length,), dtype=tf.int32)
        #tok = Input((self.length,), dtype=tf.int32)
        
        config = RobertaConfig.from_pretrained(self.config_file)
        config.output_hidden_states=True
        bert_model = TFRobertaModel.from_pretrained(self.bert_model_file,config=config)
        
        _ , _ , x = bert_model(ids,attention_mask=att)
        
        def output_layer(bert_output, name='start', activation='softmax'):
            x_bert = K.max(K.stack(bert_output), axis=0)
            x_bert = Dropout(0.5)(x_bert)
            #x_bert = Dropout(0.1)(bert_output)
            #x_bert = Conv1D(128, 2,padding='same')(x_bert)
            #x_bert = LeakyReLU()(x_bert)
            #x_bert = Conv1D(64, 2, padding='same')(x_bert)
            #x_bert = Conv1D(1, 2, padding='same')(x_bert)
            x_bert = Dense(1)(x_bert)                       
            x_output = Flatten()(x_bert)
            x_output = Activation(activation, name=name)(x_output)
             
            return x_output
        
        if self.separated_output:
            #x1_output = output_layer(x[0], name='start')
            #x2_output = output_layer(x[0], name='end')
            x1_output = output_layer([x[-1], x[-2], x[-3]], name='start')
            x2_output = output_layer([x[-1], x[-2], x[-3]], name='end')
    
            model = Model(inputs=[ids, att], outputs=[x1_output,x2_output])
            # optimizer = Adam(learning_rate=3e-5)
            # model.compile(loss='categorical_crossentropy', optimizer=optimizer)
        else:
            #x_output = output_layer(x[0], name='span', activation='sigmoid')
            x_output = output_layer([x[-1], x[-2], x[-3]], name='span', activation='sigmoid')   
            model = Model(inputs=[ids, att], outputs=x_output)

        #optimizer = Adam(learning_rate=1.5e-5)
        optimizer = Adam(learning_rate=2.4e-5)
        model.compile(loss=self.loss_fn, optimizer=optimizer)
        #model.compile(loss=jaccard_distance, optimizer=optimizer)
        
        # print(model.summary)
        # K.clear_session()
            
        return model
    
    def predict_decode(self, text_data, preds, vec_idx):
        all_selected_text = []
        ii = 0
        if self.separated_output:
            start = preds[0]
            end = preds[1]
        else:
            span = preds[0]

        for k in vec_idx:
            if self.separated_output:
                a = np.argmax(start[k,])
                b = np.argmax(end[k,])
            else:
                min_prob = np.min(span[k, ])
                max_prob = np.max(span[k, ])
                span_idx = np.where(span[k, ]>0.5*(max_prob-min_prob)+min_prob)[0]
                if len(span_idx)==0:
                    a=0
                    b=-1
                else:
                    a = span_idx[0]
                    b = span_idx[-1]
            if a>b: 
                ii = ii + 1
                st = text_data.loc[k,'text']
            else:
                text1 = " "+" ".join(text_data.loc[k,'text'].split())
                enc = self.tokenizer.encode(text1)
                st = self.tokenizer.decode(enc.ids[a-1:b])
            all_selected_text.append(st)
        print(ii)
        return all_selected_text

    def train_model(self, train, epochs=3, batch_size=32, version='v0', model_path=None, verbose=1):
        # USE verbose=1 FOR INTERACTIVE
        train = train.reset_index(drop=True)
        if self.separated_output:
            input_ids, attention_mask, start_tokens, end_tokens = self.bert_data_transform(train, train=True)
            oof_start = np.zeros((input_ids.shape[0],self.max_length))
            oof_end = np.zeros((input_ids.shape[0],self.max_length))
        else:
            input_ids, attention_mask, span_tokens = self.bert_data_transform(train, train=True)
            oof_span = np.zeros((input_ids.shape[0],self.max_length))

        jacs = [] 
                
        if self.n_splits == 1:
            rs = ShuffleSplit(n_splits=1, test_size=0.25, random_state=self.seed)
        else:
            rs = StratifiedKFold(n_splits=self.n_splits,shuffle=True,random_state=self.seed)
            
        for fold,(idxT,idxV) in enumerate(rs.split(input_ids,train.sentiment.values)):
            print('#'*25)
            print('### FOLD %i'%(fold+1))
            print('#'*25)
           
            K.clear_session()
            model = self.build_model()
            print(model.summary)

            if model_path is None:
                print('Training model...')
                #et = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')
                sv = ModelCheckpoint('%s-roberta-%i.h5'%(version,fold), monitor='val_loss', verbose=1, save_best_only=True,
                                     save_weights_only=True, mode='auto', save_freq='epoch')
                if self.separated_output:
                    targetT = [start_tokens[idxT,], end_tokens[idxT,]]
                    targetV = [start_tokens[idxV,], end_tokens[idxV,]]
                else:
                    targetT = span_tokens[idxT,]
                    targetV = span_tokens[idxV,]

                model.fit([input_ids[idxT,], attention_mask[idxT,]], targetT, 
                          epochs=epochs, batch_size=batch_size, verbose=verbose, callbacks=[sv],
                          validation_data=([input_ids[idxV,],attention_mask[idxV,]], targetV))
                #print('Loading model...')
                #model.load_weights('%s-roberta-%i.h5'%(version,fold))
            else:
                print('Loading model...')
                model.load_weights('%s/%s-roberta-%i.h5'%(model_path,version,fold))
            
            for label, idx in {'INF':idxT, 'OOF':idxV}.items():
                print('Predicting %s...'%label)
                if self.separated_output:
                    oof_start[idx,],oof_end[idx,] = model.predict([input_ids[idx,],attention_mask[idx,]],verbose=verbose)
                else:
                    oof_span[idx,] = model.predict([input_ids[idx,],attention_mask[idx,]],verbose=verbose)
                # DISPLAY FOLD JACCARD
                if self.separated_output:
                    all_selected_text = self.predict_decode(train, [oof_start, oof_end], idx)
                else:
                    all_selected_text = self.predict_decode(train, [oof_span], idx)
                all_jac = []
                for i in range(len(idx)):
                    all_jac.append(jaccard(all_selected_text[i],train.loc[idx[i],'selected_text']))
                print('>>>> FOLD %i Jaccard ='%(fold+1),np.mean(all_jac))
                print()
                
                if label == 'OOF':
                    jacs.append(np.mean(all_jac))
                                 
        print('>>>> OVERALL %i Fold CV Jaccard ='%self.n_splits,np.mean(jacs))
        
        
    def predict_test(self, test, version='v0', model_path=None, verbose=1):
        test = test.reset_index(drop=True)
        input_ids, attention_mask = self.bert_data_transform(test, train=False)
        
        if self.separated_output:
            preds_start = np.zeros((input_ids.shape[0],self.max_length))
            preds_end = np.zeros((input_ids.shape[0],self.max_length))
        else:
            preds_span = np.zeros((input_ids.shape[0],self.max_length))

        for fold in range(self.n_splits):
            K.clear_session()
            model = self.build_model()
            if model_path is None:
                model.load_weights('%s-roberta-%i.h5'%(version,fold))
            else:
                model.load_weights('%s/%s-roberta-%i.h5'%(model_path,version,fold))
            print('Predicting Test...')
            preds = model.predict([input_ids,attention_mask],verbose=verbose)
            if self.separated_output:
                preds_start += preds[0]/self.n_splits
                preds_end += preds[1]/self.n_splits
            else:
                preds_span += preds/self.n_splits
        
        if self.separated_output:                
            all_selected_text = self.predict_decode(test, [preds_start, preds_end], test.index)
        else:
            all_selected_text = self.predict_decode(test, [preds_span], test.index)

        return all_selected_text

# Load data

In [0]:
train = pd.read_csv(ROOT_PATH + '/input/tweet-sentiment-extraction/train.csv').fillna('')
test = pd.read_csv(ROOT_PATH + '/input/tweet-sentiment-extraction/test.csv').fillna('')

In [0]:
train.shape

(27481, 4)

In [0]:
train.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [0]:
dummy_jacs = []
for i in range(train.shape[0]):
  jac = jaccard(train.loc[i, 'text'], train.loc[i, 'selected_text'])
  dummy_jacs.append(jac)
np.mean(dummy_jacs)

0.589073146730252

In [0]:
test.head()

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive


# Setup

In [0]:
PATH = ROOT_PATH + '/input/tfrobert/'
tokenizer = tokenizers.ByteLevelBPETokenizer(
    vocab_file=PATH+'/roberta-base-vocab.json', 
    merges_file=PATH+'/roberta-base-merges.txt', 
    lowercase=True,
    add_prefix_space=True
)

# config = RobertaConfig.from_pretrained(PATH+'roberta-base-config.json')
# bert_model = TFRobertaModel.from_pretrained(PATH+'roberta-base-tf_model.h5',config=config)

# config = RobertaConfig.from_pretrained(PATH+'config-roberta-base.json')
# bert_model = TFRobertaModel.from_pretrained(PATH+'pretrained-roberta-base.h5',config=config)

SAMPLE_RUN = False # Set True if you just want to debug implementation. Otherwise False.
MODEL_PATH = None # Set None if you want to train a new model, otherwise specify the PATH of trained model
N_SPLITS = 10 # 1 means train validation split at portion 0.75:0.25
MAX_LEN = 128
SEP_OUTPUT = True # True: if separate predictions for start and end, Flase: predict everything at all

# Train model

In [0]:
def jaccard_distance(y_true, y_pred, smooth=1):
    y_true = tf.keras.backend.cast(y_true, dtype='float32')
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    #jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return -K.log(intersection + smooth) + K.log(sum_ - intersection + smooth)

def smoothed_crossentropy(y_true, y_pred):
    # adjust the targets for sequence bucketing
    ll = tf.shape(y_pred)[1]
    y_true = y_true[:, :ll]
    loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred,
        from_logits=False, label_smoothing=0.1)
    loss = tf.reduce_mean(loss)
    return loss

In [0]:
SEED = 43
tf.random.set_seed(SEED)
np.random.seed(SEED)

if SAMPLE_RUN:
    sample_train = train.sample(2000, random_state=SEED)
    sample_test = test.sample(3, random_state=SEED)

    sample_model = FineTuneBertForQA(tokenizer, config_file=PATH+'roberta-base-config.json', 
                                       bert_model_file=PATH+'roberta-base-tf_model.h5', 
                                       max_length=MAX_LEN, n_splits=N_SPLITS, loss_fn="categorical_crossentropy", separated_output=SEP_OUTPUT, seed=SEED)
    sample_model.train_model(sample_train, epochs=2, batch_size=32, version='v1', model_path=MODEL_PATH, verbose=1)
    
    sample_selected_text = sample_model.predict_test(sample_test, version='v1', model_path=MODEL_PATH, verbose=1)
    sample_test['selected_text'] = sample_selected_text
    pd.set_option('max_colwidth', 60)
    print(sample_test)
else:
    full_model = FineTuneBertForQA(tokenizer, config_file=PATH+'config-roberta-base.json', 
                                     bert_model_file=PATH+'pretrained-roberta-base.h5', 
                                     max_length=MAX_LEN, n_splits=N_SPLITS, loss_fn="categorical_crossentropy", separated_output=SEP_OUTPUT, seed=SEED)
    full_model.train_model(train, epochs=4, batch_size=16, version='v4', model_path=MODEL_PATH, verbose=1)

    all_selected_text = full_model.predict_test(test, version='v4', model_path=MODEL_PATH, verbose=1)
    test['selected_text'] = all_selected_text
    test[['textID','selected_text']].to_csv('submission.csv',index=False)
    pd.set_option('max_colwidth', 60)
    print(test.sample(25))

#########################
### FOLD 1
#########################
<bound method Network.summary of <tensorflow.python.keras.engine.training.Model object at 0x7f5a90273da0>>
Training model...
Epoch 1/4
1546/1546 [==============================] - ETA: 0s - loss: 2.3122 - start_loss: 1.1320 - end_loss: 1.1802
Epoch 00001: val_loss improved from inf to 1.69637, saving model to v4-roberta-0.h5
1546/1546 [==============================] - 417s 270ms/step - loss: 2.3122 - start_loss: 1.1320 - end_loss: 1.1802 - val_loss: 1.6964 - val_start_loss: 0.8718 - val_end_loss: 0.8246
Epoch 2/4
1546/1546 [==============================] - ETA: 0s - loss: 1.7587 - start_loss: 0.8833 - end_loss: 0.8754
Epoch 00002: val_loss improved from 1.69637 to 1.68213, saving model to v4-roberta-0.h5
1546/1546 [==============================] - 414s 268ms/step - loss: 1.7587 - start_loss: 0.8833 - end_loss: 0.8754 - val_loss: 1.6821 - val_start_loss: 0.8560 - val_end_loss: 0.8261
Epoch 3/4
1546/1546 [=================

In [0]:
all_selected_text = full_model.predict_test(train, version='v3', model_path=MODEL_PATH, verbose=1)
train['pred_selected_text'] = all_selected_text

Predicting Test...
859/859 [==============================] - 138s 160ms/step
Predicting Test...
859/859 [==============================] - 138s 161ms/step
Predicting Test...
859/859 [==============================] - 137s 160ms/step
Predicting Test...
859/859 [==============================] - 137s 160ms/step
Predicting Test...
859/859 [==============================] - 137s 159ms/step
48


In [0]:
pred_jacs = []
for i in range(train.shape[0]):
  jac = jaccard(train.loc[i, 'pred_selected_text'], train.loc[i, 'selected_text'])
  pred_jacs.append(jac)
print(np.mean(pred_jacs))
check_pred_jac =pd.DataFrame({'sentiment':train['sentiment'],'pred_jaccard':pred_jacs})
check_pred_jac.groupby(['sentiment'])['pred_jaccard'].mean()

0.749140919752366


sentiment
negative    0.600322
neutral     0.977581
positive    0.588125
Name: pred_jaccard, dtype: float64

In [0]:
a = tf.constant([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
b = tf.constant([[10, 20, 30], [40, 50, 60], [70, 80, 90]])

In [0]:
K.mean(K.stack((a, b)), axis=0)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 5, 11, 16],
       [22, 27, 33],
       [38, 44, 49]], dtype=int32)>

In [0]:
import torch

In [0]:
a = torch.tensor([[1., 2, 3], [4, 5, 6], [7, 8, 9]])
b = torch.tensor([[10., 20, 30], [40, 50, 60], [70, 80, 90]])
torch.mean(torch.stack([a, b]), 0)

tensor([[ 5.5000, 11.0000, 16.5000],
        [22.0000, 27.5000, 33.0000],
        [38.5000, 44.0000, 49.5000]])